##### Colab

In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import userdata
import os

os.environ["kaggle_key"] = userdata.get('kaggle_key')
os.environ["kaggle_username"] = userdata.get('kaggle_username')

In [ ]:
!kaggle datasets download -d ravidussilva/real-ai-art
! unzip "real-ai-art"

##### Local & Colab

In [ ]:
#Imports
import sys
sys.path.append('/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024')
#sys.path.append('/content/drive/MyDrive/06_TUWien/10_ADL')
from scripts.run_cnn import train_model, test_model
from scripts.viz import display_images

from datasets.AIArtBench import AIArtbench
from datasets.dataset import Subset
from datasets.preprocessing import CustomDatasetPreprocessor
from torch.utils.data import DataLoader
from models.cnn import CNN_Net
from models.simple_cnn import Simple_CNN
from scripts.evaluation import analyze_test_results
import torchvision.transforms as transforms

#### Import dataset & look what's inside

In [3]:
# Define your dataset path
dataset_path = "/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024/data"
#dataset_path = "/content/Real_AI_SD_LD_Dataset"

In [3]:
# Process Data to print some images
preprocessor = CustomDatasetPreprocessor(dataset_path)
preprocessor.preprocess(fraction=0.2)

# Get dataset splits
train_data, validation_data, test_data = preprocessor.get_splits()

In [ ]:
# Print images from train dataset + check how different transformations work

train_transform = transforms.Compose([
        transforms.Resize(size=(256, 256)),
        transforms.ToTensor()
    ])

# Prepare the training dataset and DataLoader
train_dataset = AIArtbench(dataframe=train_data, subset=Subset.TRAINING, transform=train_transform)
# Extract images
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, )
# Get unique class names
classes = train_dataset.get_classes()
# Display a few images from the training data
display_images(data_loader=train_data_loader, classes=classes, num_images=5, save_path="train_images_grid.png")

#### Train & test baseline (Simple CNN)

In [6]:
# Define transformations

train_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])

val_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])

test_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])


# Config for training and testing
config = {
    'model': Simple_CNN(),
    'dataset_path': dataset_path,
    'model_save_dir': 'saved_models',
    'results_save_dir': 'saved_results',
    'train_transform': train_transform,
    'val_transform': val_transform,
    'test_transform': test_transform,
    'batch_size': 64,
    'learning_rate': 0.001,
    'scheduler_gamma': 0.9,
    'epochs': 10,
    'fraction': 1,
    'val_frequency': 2,
    'debug_mode': True,
    'patience': 3,
    'model_name': "Simple_CNN"
}

# Train the model

#trainer = train_model(config)

In [ ]:
# Test the model (reuse the trainer)
test_loss, test_accuracy, test_per_class_accuracy, all_labels, all_predictions, test_classes = test_model(config, trainer=None)

In [ ]:
# Analyze test results
metrics = analyze_test_results(
    test_loss=test_loss,
    test_accuracy=test_accuracy,
    test_per_class_accuracy=test_per_class_accuracy,
    all_labels=all_labels,
    all_predictions=all_predictions,
    classes=test_classes
)

#### Train & test using transfer learning (ResNet18)

In [ ]:
from models.resnet18 import ResNet18FineTuned

train_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])

val_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])

test_transform = transforms.Compose([
        transforms.Resize(size=(32, 32)),
        transforms.ToTensor()
    ])
#freeze_layers=True & freeze_layers = ['conv1', 'bn1', 'layer1', 'layer2'] - what layers need to be frozen
#freeze_layers=False - do not freeze any layers
#freeze_layers=True - freeze all layers except for the last one
freeze_layers = ['conv1', 'bn1', 'layer1', 'layer2']
# Config for training and testing
config_resnet = {
    'model': ResNet18FineTuned(freeze_layers=True, specific_layers_to_freeze=freeze_layers),
    'dataset_path': dataset_path,
    'model_save_dir': 'saved_models',
    'results_save_dir': 'saved_results',
    'train_transform': train_transform,
    'val_transform': val_transform,
    'test_transform': test_transform,
    'batch_size': 64,
    'learning_rate': 0.001,
    'scheduler_gamma': 0.9,
    'epochs': 10,
    'fraction': 0.01,
    'val_frequency': 2,
    'debug_mode': True,
    'patience': 3,
    'model_name': "ResNet18"
}

# Train the model

trainer_resnet = train_model(config_resnet)
